In [1]:
import os
import docx
from docx import Document
import re
import shutil
import zipfile
import xml.etree.ElementTree as ET
import aspose.words as aw
from xml.etree.ElementTree import ParseError
import json
import pandas as pd

In [2]:
# LEER RUTA DE ARCHIVO tipo A
docx_path = 'AI-Redações/RedaçõesDownloaded/R1/Ana Luiza Ferreira Braulio A.docx'
docx_path1 = 'AI-Redações/RedaçõesDownloaded/R1/Claudio Gabriel A11.docx'
docx_path2 = 'AI-Redações/RedaçõesDownloaded/R1/Érico Breyer de Freitas A.docx'

In [3]:
# LEER RUTA DE ARCHIVO tipo Z
docx_pathZ = 'AI-Redações/RedaçõesDownloaded/R1/DIOGO DAVID SÁNCHEZ LIMA Z.docx'
docx_path1Z = 'AI-Redações/RedaçõesDownloaded/R1/Davi Natael da Cunha  ZZ.docx'
docx_path2Z = 'AI-Redações/RedaçõesDownloaded/R1/Carallow ZZ.docx'

In [39]:
def extraer_xml_de_docx(docx_path, output_folder): #CREA CARPETA
    # Asegurarse de que la carpeta de salida existe
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    with zipfile.ZipFile(docx_path, 'r') as zip_ref:
        zip_ref.extractall(output_folder)
        
def extraer_texto_LIMPIO(docx_path):
    doc = docx.Document(docx_path)
    texto_completo = []
    for para in doc.paragraphs:
        texto_completo.append(para.text)
    return '\n'.join(texto_completo)

def todo(docx_path):
    docToRead = aw.Document(docx_path)
    TEXTO = []
    for paragraph in docToRead.get_child_nodes(aw.NodeType.PARAGRAPH, True) :    
        paragraph = paragraph.as_paragraph()
        TEXTO.append(paragraph.to_string(aw.SaveFormat.TEXT))
    return TEXTO

#EXTRAE LIMPIO
def extraer_texto_en_parrafosTOTAL_SINEXCLUIDAS(docx_path):
    with zipfile.ZipFile(docx_path) as docx:
        xml_content = docx.read('word/document.xml')
    try:
        tree = ET.ElementTree(ET.fromstring(xml_content))
    except ParseError:
        return "Error al analizar el XML"
    parrafos = []
    for elem in tree.iter():
        if elem.tag.endswith('}p'):
            parrafo = []
            for subelem in elem.iter():
                if subelem.tag.endswith('}t') and subelem.text:
                    parrafo.append(subelem.text)
            parrafos.append(''.join(parrafo))
    return parrafos

def extraer_comentarios(docx_path):
    
    with zipfile.ZipFile(docx_path) as docx:
        if zipfile.ZipFile(docx_path) != {}:
            comments_xml = docx.read('word/comments.xml')
            tree = ET.fromstring(comments_xml)
            comments = []
            namespace = '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}'
            for comment in tree.iter(namespace + 'comment'):
                text = ''.join(t.text for t in comment.iter(namespace + 't') if t.text)
                comments.append(text)
        else:
            comments = []
  
            
    #tree = ET.fromstring(comments_xml)
    #comments = []
    #namespace = '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}'
    #for comment in tree.iter(namespace + 'comment'):
    #    text = ''.join(t.text for t in comment.iter(namespace + 't') if t.text)
    #    comments.append(text)
    return comments


In [5]:
def encontrar_comment_range_start(comentario):
    # Recorrer hacia atrás desde el comentario hasta encontrar CommentRangeStart
    nodo_actual = comentario.previous_sibling
    while nodo_actual is not None:
        if nodo_actual.node_type == aw.NodeType.COMMENT_RANGE_START:
            return nodo_actual
        nodo_actual = nodo_actual.previous_sibling
    return None

def encontrar_comment_range_end(comentario):
    # Recorrer hacia adelante desde el comentario hasta encontrar CommentRangeEnd
    nodo_actual = comentario.previous_sibling #next_sibling
    while nodo_actual is not None:
        if nodo_actual.node_type == aw.NodeType.COMMENT_RANGE_END:
            return nodo_actual
        nodo_actual = nodo_actual.next_sibling
    return None

def extraer_texto_entre_nodos(doc, nodo_inicio, nodo_fin):
    # Extraer el texto entre dos nodos
    texto = []
    if nodo_inicio is not None:
        nodo_actual = nodo_inicio.next_sibling
        while nodo_actual is not None and nodo_actual != nodo_fin:
            if nodo_actual.node_type == aw.NodeType.RUN:
                texto.append(nodo_actual.get_text())
            nodo_actual = nodo_actual.next_sibling
    return ''.join(texto)

In [6]:
def obtener_comentarios_y_texto_asociado(documento):
    # Cargar el documento
    doc = aw.Document(documento)

    # Lista para almacenar pares de comentario y texto asociado
    comentarios_y_texto = []

    # Obtener todos los comentarios
    comentarios = doc.get_child_nodes(aw.NodeType.COMMENT, True)

    for comentario in comentarios:
        comentario = comentario.as_comment()

        # Encontrar el CommentRangeStart más cercano
        comment_range_start = encontrar_comment_range_start(comentario)

        # Encontrar el CommentRangeEnd más cercano
        comment_range_end = encontrar_comment_range_end(comentario)

        # Extraer el texto asociado con este comentario
        texto_asociado = extraer_texto_entre_nodos(doc, comment_range_start, comment_range_end)

        comentarios_y_texto.append((comentario.get_text().strip(), texto_asociado))

    return comentarios_y_texto

In [7]:
#CREAR VARIABLE JSON

In [8]:
#CREAR VARIABLE JSON PARA DOCUMENTOS TIPO "A"

def JSONTipoA(docx_path):
    texto_LIMPIO = extraer_texto_LIMPIO(docx_path) #extraer_texto_en_parrafosTOTAL_SINEXCLUIDAS
    texto_LIMPIO = texto_LIMPIO.splitlines()
    texto_LIMPIO = [elemento for elemento in texto_LIMPIO if elemento != ""] 
    #NOME
    nome = texto_LIMPIO[0].replace("Nome: ", "").replace("Nome:", "").replace("Aluno: ", "").replace("Aluno:", "")
    del texto_LIMPIO[0]
    #CORREO
    correo = texto_LIMPIO[0].replace("E-mail: ", "").replace("E-mail:", "").replace("Email: ", "").replace("Email:", "").replace("\xa0", "")
    del texto_LIMPIO[0]
    #CODIGO
    codigo = texto_LIMPIO[0].replace("Código de identificação: ", "").replace("Código de identificação:", "")
    del texto_LIMPIO[0]
    #NOTA
    nota = texto_LIMPIO[-1].replace("NOTA: ", "").replace("nota: ", "")
    del texto_LIMPIO[-1]
    
    print("nome",nome)
    print("texto_LIMPIO", len(texto_LIMPIO))
    indice = next((i for i in reversed(range(len(texto_LIMPIO))) if nome.split()[0] in texto_LIMPIO[i] or nome.split()[0] + " " + nome.split()[1] in texto_LIMPIO[i]), None)
    print("indice",indice)
    
    #if indice is not None:
    cuerpo = texto_LIMPIO[:indice]
    conclusion = texto_LIMPIO[indice:]        
    comentarios_y_texto = obtener_comentarios_y_texto_asociado(docx_path)
    comentarios_texto = [(tupla[0].replace('\x05', ''), tupla[1]) for tupla in comentarios_y_texto]

    datos = {
        "NOMBRE": nome,
        "CORREO": correo,
        "CODIGO": codigo,
        "CUERPO": cuerpo,
        "COMENTARIOS": comentarios_texto,
        "CONCLUSION": {
            "NOTA": nota,
            "TEXT": conclusion
        }
    }
    # Convertir el diccionario a una cadena JSON
    json_str = json.dumps(datos, ensure_ascii=False, indent=4)
    return json_str

In [9]:
docx_path

'AI-Redações/RedaçõesDownloaded/R1/Ana Luiza Ferreira Braulio A.docx'

In [ ]:
JSON = JSONTipoA(docx_path)

In [ ]:
print(JSON)

In [ ]:
ruta = "/home/andres/Documents/analisisTexto/AI-Redações/RedaçõesDownloaded/R1"

# Dividir la ruta por cada "/"
partes = ruta.split("/")

# Seleccionar la última parte
ultima_parte = partes[-1]

print(ultima_parte)

In [ ]:
import os

In [ ]:
ruta_carpeta1 = '/home/andres/Documents/analisisTexto/AI-Redações/RedaçõesDownloaded/'
archivos_filtrados1 = []
for archivo in os.listdir(ruta_carpeta1):
    archivos_filtrados1.append(archivo)
print(archivos_filtrados1)

In [ ]:
archivos_filtradosA, archivos_filtradosZ, archivos_filtradosR = [],[],[]

for i in archivos_filtrados1:
    ruta_archivo = ruta_carpeta1 + str(i)
    #print(ruta_archivo)
    
    #for j in ruta_archivo:
            # Recorre todos los archivos en el directorio
    for archivo in os.listdir(ruta_archivo):
            #print(archivo)
            if archivo.endswith('A.docx') or archivo.endswith('A1.docx') or archivo.endswith('A11.docx') or archivo.endswith('A2.docx') or archivo.endswith('A3.docx'):
                dic = {
                    "id":i,
                    "archivo":archivo  
                }
                archivos_filtradosA.append(dic)
            
            elif archivo.endswith('Z.docx') or archivo.endswith('Z1.docx') or archivo.endswith('Z11.docx') or archivo.endswith('Z2.docx'):
                dic1 = {
                    "id":i,
                    "archivo":archivo  
                }
                archivos_filtradosZ.append(dic1) 
            
            elif archivo.endswith('R.docx') or archivo.endswith('R1.docx') or archivo.endswith('R2.docx') or archivo.endswith('R3.docx') or archivo.endswith('RR12.docx'):
                dic2 = {
                    "id":i,
                    "archivo":archivo  
                }
                archivos_filtradosR.append(dic2) 
        

In [ ]:
len(archivos_filtradosA), len(archivos_filtradosZ), len(archivos_filtradosR)


In [ ]:
r = 'AI-Redações/RedaçõesDownloaded/'
for i in archivos_filtradosA:
    i["JSON"] = JSONTipoA(r+str(i["id"])+"/"+str(i["archivo"]))
    #JSON = JSONTipoA(r+str(i))

In [ ]:
import json

# Tu diccionario
#mi_diccionario = {'clave1': 'valor1', 'clave2': 'valor2'}

# Escribir el diccionario a un archivo JSON
with open('archivos_filtradosA.json', 'w') as archivo_json:
    json.dump(archivos_filtradosA, archivo_json)

In [ ]:
print(archivos_filtradosA[1]['JSON'])

In [ ]:
len(archivos_filtrados)

In [ ]:
diccionario = {}
Lid,Larchivo,Ltexto,Lcomentario,Lpalabras_corregidas = [],[],[],[],[]
tabla = pd.DataFrame()
for j in range(len(archivos_filtradosA)):
    data = json.loads(archivos_filtradosA[j]['JSON'])
    #data["COMENTARIOS"]
    for i in range(len(data["COMENTARIOS"])):
        print(data["COMENTARIOS"][i][1])
        #tabla["texto"] = [data["COMENTARIOS"][i][1]]
        #texto = data["COMENTARIOS"][i][1]
        Lid.append(archivos_filtradosA[j]["id"])
        Larchivo.append(archivos_filtradosA[j]['archivo'])
        Ltexto.append(data["COMENTARIOS"][i][1])
        Lcomentario.append(data["COMENTARIOS"][i][0])
        Lpalabras_corregidas.append(data["PALABRAS_CORREGIDAS"])
        #comentario = data["COMENTARIOS"][i][0]
        print(data["COMENTARIOS"][i][0])
        #diccionario["texto"] = texto
        #diccionario["comentario"] = comentario
        #tabla["comentario"] = [data["COMENTARIOS"][i][0]]

In [ ]:
datos = {
    "id":Lid,
    "archivo":Larchivo,
    "texto": Ltexto,
    "comentario":Lcomentario,
    "palabras_corregidas":Lpalabras_corregidas
}
df = pd.DataFrame(datos)

In [ ]:
df

In [ ]:
df.to_excel("archivo.xlsx", index=False)

## TIPO Z

In [ ]:
print(docx_pathZ)
print(docx_path1Z)

In [ ]:
def JSONTipoZ(docx_pathZ):     
    texto_LIMPIO = extraer_texto_LIMPIO(docx_pathZ) #SIN NOTE Y TEXT
    texto_LIMPIO = texto_LIMPIO.split('\n')
    texto_LIMPIO = [elemento for elemento in texto_LIMPIO if elemento != ""] 
    texto_LIMPIO = [elemento.replace("\xa0", "") for elemento in texto_LIMPIO]
    texto_LIMPIO = [elemento for elemento in texto_LIMPIO if elemento != ""]
    
    a = extraer_texto_en_parrafosTOTAL_SINEXCLUIDAS(docx_pathZ)
    a = [elemento for elemento in a if elemento != ""] 
    a = [elemento.replace("\xa0", "") for elemento in a]
    a = [elemento for elemento in a if elemento != ""] 
    
    b = todo(docx_pathZ)
    #com = obtener_comentarios_y_texto_asociado(docx_pathZ)
    
    #comentarios_y_texto = obtener_comentarios_y_texto_asociado(docx_pathZ)
    #comentarios_texto = [(tupla[0].replace('\x05', ''), tupla[1]) for tupla in comentarios_y_texto]
    com = extraer_comentarios(docx_pathZ)
    #PREPROCESAMIENTO
    mi_lista_limpia1 = [elemento.replace("\xa0", " ") for elemento in b]
    mi_lista_limpia1 = [elemento.replace("\r\n", "") for elemento in mi_lista_limpia1]
    mi_lista_limpia1 = [elemento for elemento in mi_lista_limpia1 if elemento != ""]
    mi_lista_limpia1 = [elemento for elemento in mi_lista_limpia1 if elemento != " "]
    mi_lista_limpia1 = mi_lista_limpia1[1:-1]
    resultado = [elemento for elemento in mi_lista_limpia1 if elemento not in com]
    
    #print(comentarios_texto)
    for i in com:
        resultado3 = [elemento.replace(i,"") for elemento in resultado]
        
    un = a[-(len(a)-len(texto_LIMPIO)):]
    nota = un[-1]
    conclusion = un[:-1]
    #print(nota), print(conclusion)
    
    #NOME
    nome = texto_LIMPIO[0].replace("Nome: ", "").replace("Nome:", "").replace("Aluno: ", "").replace("Aluno:", "")
    del texto_LIMPIO[0]
    #CORREO
    correo = texto_LIMPIO[0].replace("E-mail: ", "").replace("E-mail:", "").replace("Email: ", "").replace("Email:", "").replace("\xa0", "")
    del texto_LIMPIO[0]
    #CODIGO
    codigo = texto_LIMPIO[0].replace("Código de identificação: ", "").replace("Código de identificação:", "")
    del texto_LIMPIO[0]
    #NOTA
    
    #nota = texto_LIMPIO[-1].replace("NOTA: ", "").replace("nota: ", "").replace("Nota ", "")
    #del texto_LIMPIO[-1]
    comentarios_y_texto = obtener_comentarios_y_texto_asociado(docx_pathZ)
    comentarios_texto = [(tupla[0].replace('\x05', ''), tupla[1]) for tupla in comentarios_y_texto]
    
    t2 = a[:len(texto_LIMPIO)]
    t2 = t2[3:]
    
    resultado1 = resultado[:len(texto_LIMPIO)]
    resultado1 = resultado1[3:]
    
    cuerpo = t2
    
    # OBTENER PALABRAS COMPLETA CON ERROR
    PALABRAS_CON_ERROR = []
    for s1, s2 in zip(resultado1, t2):
        palabras_s1 = s1.split()
        palabras_s2 = s2.split()
        palabras_restantes = [palabra for palabra in palabras_s1 if palabra not in palabras_s2]
        frase_resultante = ' '.join(palabras_restantes)
        PALABRAS_CON_ERROR.extend(frase_resultante.split())
    
    #print(PALABRAS_CON_ERROR) 
    
    # OBTENER PALABRAS LIMPIAS CORREGIDAS
    PALABRAS_CORREGIDO = []
    for s1, s2 in zip(t2, resultado1):
        palabras_s1 = s1.split()
        palabras_s2 = s2.split()
        palabras_restantes = [palabra for palabra in palabras_s1 if palabra not in palabras_s2]
        frase_resultante = ' '.join(palabras_restantes)
        PALABRAS_CORREGIDO.extend(frase_resultante.split())
    
    #print(PALABRAS_CORREGIDO) 
    palabras_corregidas = [PALABRAS_CON_ERROR,PALABRAS_CORREGIDO]
    
    datos = {
        "NOMBRE": nome,
        "CORREO": correo,
        "CODIGO": codigo,
        "CUERPO": cuerpo,
        "COMENTARIOS": comentarios_texto,
        "PALABRAS_CORREGIDAS":palabras_corregidas,
        "CONCLUSION": {
            "NOTA": nota,
            "TEXT": conclusion
        }
    }
    json_str = json.dumps(datos, ensure_ascii=False, indent=4)
    return json_str

In [ ]:
comentarios_y_texto = obtener_comentarios_y_texto_asociado(docx_pathZ)
comentarios_texto = [(tupla[0].replace('\x05', ''), tupla[1]) for tupla in comentarios_y_texto]
comentarios_texto

In [ ]:
F = JSONTipoZ(docx_pathZ)

In [ ]:
print(F)


In [ ]:
archivos_filtradosZ1 = archivos_filtradosZ

In [ ]:
archivos_filtradosZ[0]["archivo"]

In [ ]:
r = 'AI-Redações/RedaçõesDownloaded/' #modificar el algoritmo si es q no hay comentarios
for i in archivos_filtradosZ1:
    print(i["id"],i["archivo"])
    i["JSON"] = JSONTipoZ(r+str(i["id"])+"/"+str(i["archivo"]))
    #JSON = JSONTipoA(r+str(i))

In [ ]:
archivos_filtradosZ1

In [ ]:
len(archivos_filtradosZ1)

In [ ]:
print(archivos_filtradosZ[10]["JSON"])

In [ ]:
diccionario = {}
Lid,Larchivo,Ltexto,Lcomentario,Lpalabras_corregidas = [],[],[],[],[]
tabla = pd.DataFrame()
for j in range(len(archivos_filtradosZ)):
    data = json.loads(archivos_filtradosZ[j]['JSON'])
    #data["COMENTARIOS"]
    for i in range(len(data["COMENTARIOS"])):
        print(data["COMENTARIOS"][i][1])
        #tabla["texto"] = [data["COMENTARIOS"][i][1]]
        #texto = data["COMENTARIOS"][i][1]
        Lid.append(archivos_filtradosZ[j]["id"])
        Larchivo.append(archivos_filtradosZ[j]['archivo'])
        Ltexto.append(data["COMENTARIOS"][i][1])
        Lcomentario.append(data["COMENTARIOS"][i][0])
        Lpalabras_corregidas.append(data["PALABRAS_CORREGIDAS"])
        #comentario = data["COMENTARIOS"][i][0]
        print(data["COMENTARIOS"][i][0])
        #diccionario["texto"] = texto
        #diccionario["comentario"] = comentario
        #tabla["comentario"] = [data["COMENTARIOS"][i][0]]

In [ ]:
datos = {
    "id":Lid,
    "archivo":Larchivo,
    "texto": Ltexto,
    "comentario":Lcomentario,
    "palabras_corregidas":Lpalabras_corregidas
}
df = pd.DataFrame(datos)

In [ ]:
Lpalabras_corregidas

In [ ]:
df.head(20)

In [ ]:
del df["palabras_corregidas"]

In [ ]:
df.to_excel("archivoZ.xlsx", index=False)

## TIPO R

In [ ]:
docx_pathR = "AI-Redações/RedaçõesDownloaded/R2/Erick Vinicius R.docx"

In [ ]:
docx_pathR

In [ ]:
def JSONTipoR(docx_pathR):     
    texto_LIMPIO = extraer_texto_LIMPIO(docx_pathR) #SIN NOTE Y TEXT
    texto_LIMPIO = texto_LIMPIO.split('\n')
    texto_LIMPIO = [elemento for elemento in texto_LIMPIO if elemento != ""] 
    texto_LIMPIO = [elemento.replace("\xa0", "") for elemento in texto_LIMPIO]
    texto_LIMPIO = [elemento for elemento in texto_LIMPIO if elemento != ""]
    
    a = extraer_texto_en_parrafosTOTAL_SINEXCLUIDAS(docx_pathR)
    a = [elemento for elemento in a if elemento != ""] 
    a = [elemento.replace("\xa0", "") for elemento in a]
    a = [elemento for elemento in a if elemento != ""] 
    
    b = todo(docx_pathR)
    #com = obtener_comentarios_y_texto_asociado(docx_pathZ)
    
    #comentarios_y_texto = obtener_comentarios_y_texto_asociado(docx_pathZ)
    #comentarios_texto = [(tupla[0].replace('\x05', ''), tupla[1]) for tupla in comentarios_y_texto]
    com = extraer_comentarios(docx_pathR)
    #PREPROCESAMIENTO
    mi_lista_limpia1 = [elemento.replace("\xa0", " ") for elemento in b]
    mi_lista_limpia1 = [elemento.replace("\r\n", "") for elemento in mi_lista_limpia1]
    mi_lista_limpia1 = [elemento for elemento in mi_lista_limpia1 if elemento != ""]
    mi_lista_limpia1 = [elemento for elemento in mi_lista_limpia1 if elemento != " "]
    mi_lista_limpia1 = mi_lista_limpia1[1:-1]
    resultado = [elemento for elemento in mi_lista_limpia1 if elemento not in com]
    
    #print(comentarios_texto)
    for i in com:
        resultado3 = [elemento.replace(i,"") for elemento in resultado]
        
    un = a[-(len(a)-len(texto_LIMPIO)):]
    nota = un[-1]
    conclusion = un[:-1]
    #print(nota), print(conclusion)
    
    #NOME
    nome = texto_LIMPIO[0].replace("Nome: ", "").replace("Nome:", "").replace("Aluno: ", "").replace("Aluno:", "")
    del texto_LIMPIO[0]
    #CORREO
    correo = texto_LIMPIO[0].replace("E-mail: ", "").replace("E-mail:", "").replace("Email: ", "").replace("Email:", "").replace("\xa0", "")
    del texto_LIMPIO[0]
    #CODIGO
    codigo = texto_LIMPIO[0].replace("Código de identificação: ", "").replace("Código de identificação:", "")
    del texto_LIMPIO[0]
    #NOTA
    
    #nota = texto_LIMPIO[-1].replace("NOTA: ", "").replace("nota: ", "").replace("Nota ", "")
    #del texto_LIMPIO[-1]
    comentarios_y_texto = obtener_comentarios_y_texto_asociado(docx_pathR)
    comentarios_texto = [(tupla[0].replace('\x05', ''), tupla[1]) for tupla in comentarios_y_texto]
    
    t2 = a[:len(texto_LIMPIO)]
    t2 = t2[3:]
    
    resultado1 = resultado[:len(texto_LIMPIO)]
    resultado1 = resultado1[3:]
    
    cuerpo = t2
    
    # OBTENER PALABRAS COMPLETA CON ERROR
    PALABRAS_CON_ERROR = []
    for s1, s2 in zip(resultado1, t2):
        palabras_s1 = s1.split()
        palabras_s2 = s2.split()
        palabras_restantes = [palabra for palabra in palabras_s1 if palabra not in palabras_s2]
        frase_resultante = ' '.join(palabras_restantes)
        PALABRAS_CON_ERROR.extend(frase_resultante.split())
    
    #print(PALABRAS_CON_ERROR) 
    
    # OBTENER PALABRAS LIMPIAS CORREGIDAS
    PALABRAS_CORREGIDO = []
    for s1, s2 in zip(t2, resultado1):
        palabras_s1 = s1.split()
        palabras_s2 = s2.split()
        palabras_restantes = [palabra for palabra in palabras_s1 if palabra not in palabras_s2]
        frase_resultante = ' '.join(palabras_restantes)
        PALABRAS_CORREGIDO.extend(frase_resultante.split())
    
    #print(PALABRAS_CORREGIDO) 
    palabras_corregidas = [PALABRAS_CON_ERROR,PALABRAS_CORREGIDO]
    
    datos = {
        "NOMBRE": nome,
        "CORREO": correo,
        "CODIGO": codigo,
        "CUERPO": cuerpo,
        "COMENTARIOS": comentarios_texto,
        "PALABRAS_CORREGIDAS":palabras_corregidas,
        "CONCLUSION": {
            "NOTA": nota,
            "TEXT": conclusion
        }
    }
    json_str = json.dumps(datos, ensure_ascii=False, indent=4)
    return json_str

In [ ]:
F = JSONTipoR(docx_pathR)

In [ ]:
print(F)

## TIPO R

In [ ]:
archivos_filtradosR[0]["archivo"]

In [ ]:
archivos_filtradosR

In [ ]:
docx_pathR = 'AI-Redações/RedaçõesDownloaded/R12/'+archivos_filtradosR[0]["archivo"]

In [ ]:
def JSONTipoR(docx_pathR):     
    texto_LIMPIO = extraer_texto_LIMPIO(docx_pathR) #SIN NOTE Y TEXT
    texto_LIMPIO = texto_LIMPIO.split('\n')
    texto_LIMPIO = [elemento for elemento in texto_LIMPIO if elemento != ""] 
    texto_LIMPIO = [elemento.replace("\xa0", "") for elemento in texto_LIMPIO]
    texto_LIMPIO = [elemento for elemento in texto_LIMPIO if elemento != ""]
    
    a = extraer_texto_en_parrafosTOTAL_SINEXCLUIDAS(docx_pathR)
    a = [elemento for elemento in a if elemento != ""] 
    a = [elemento.replace("\xa0", "") for elemento in a]
    a = [elemento for elemento in a if elemento != ""] 
    
    b = todo(docx_pathR)
    #com = obtener_comentarios_y_texto_asociado(docx_pathZ)
    
    #comentarios_y_texto = obtener_comentarios_y_texto_asociado(docx_pathZ)
    #comentarios_texto = [(tupla[0].replace('\x05', ''), tupla[1]) for tupla in comentarios_y_texto]
    com = extraer_comentarios(docx_pathR)
    #PREPROCESAMIENTO
    mi_lista_limpia1 = [elemento.replace("\xa0", " ") for elemento in b]
    mi_lista_limpia1 = [elemento.replace("\r\n", "") for elemento in mi_lista_limpia1]
    mi_lista_limpia1 = [elemento for elemento in mi_lista_limpia1 if elemento != ""]
    mi_lista_limpia1 = [elemento for elemento in mi_lista_limpia1 if elemento != " "]
    mi_lista_limpia1 = mi_lista_limpia1[1:-1]
    resultado = [elemento for elemento in mi_lista_limpia1 if elemento not in com]
    
    #print(comentarios_texto)
    for i in com:
        resultado3 = [elemento.replace(i,"") for elemento in resultado]
        
    un = a[-(len(a)-len(texto_LIMPIO)):]
    nota = un[-1]
    conclusion = un[:-1]
    #print(nota), print(conclusion)
    
    #NOME
    nome = texto_LIMPIO[0].replace("Nome: ", "").replace("Nome:", "").replace("Aluno: ", "").replace("Aluno:", "")
    del texto_LIMPIO[0]
    #CORREO
    correo = texto_LIMPIO[0].replace("E-mail: ", "").replace("E-mail:", "").replace("Email: ", "").replace("Email:", "").replace("\xa0", "")
    del texto_LIMPIO[0]
    #CODIGO
    codigo = texto_LIMPIO[0].replace("Código de identificação: ", "").replace("Código de identificação:", "")
    del texto_LIMPIO[0]
    #NOTA
    
    #nota = texto_LIMPIO[-1].replace("NOTA: ", "").replace("nota: ", "").replace("Nota ", "")
    #del texto_LIMPIO[-1]
    comentarios_y_texto = obtener_comentarios_y_texto_asociado(docx_pathR)
    comentarios_texto = [(tupla[0].replace('\x05', ''), tupla[1]) for tupla in comentarios_y_texto]
    
    t2 = a[:len(texto_LIMPIO)]
    t2 = t2[3:]
    
    resultado1 = resultado[:len(texto_LIMPIO)]
    resultado1 = resultado1[3:]
    
    cuerpo = t2
    
    # OBTENER PALABRAS COMPLETA CON ERROR
    PALABRAS_CON_ERROR = []
    for s1, s2 in zip(resultado1, t2):
        palabras_s1 = s1.split()
        palabras_s2 = s2.split()
        palabras_restantes = [palabra for palabra in palabras_s1 if palabra not in palabras_s2]
        frase_resultante = ' '.join(palabras_restantes)
        PALABRAS_CON_ERROR.extend(frase_resultante.split())
    
    #print(PALABRAS_CON_ERROR) 
    
    # OBTENER PALABRAS LIMPIAS CORREGIDAS
    PALABRAS_CORREGIDO = []
    for s1, s2 in zip(t2, resultado1):
        palabras_s1 = s1.split()
        palabras_s2 = s2.split()
        palabras_restantes = [palabra for palabra in palabras_s1 if palabra not in palabras_s2]
        frase_resultante = ' '.join(palabras_restantes)
        PALABRAS_CORREGIDO.extend(frase_resultante.split())
    
    #print(PALABRAS_CORREGIDO) 
    palabras_corregidas = [PALABRAS_CON_ERROR,PALABRAS_CORREGIDO]
    
    datos = {
        "NOMBRE": nome,
        "CORREO": correo,
        "CODIGO": codigo,
        "CUERPO": cuerpo,
        "COMENTARIOS": comentarios_texto,
        "PALABRAS_CORREGIDAS":palabras_corregidas,
        "CONCLUSION": {
            "NOTA": nota,
            "TEXT": conclusion
        }
    }
    json_str = json.dumps(datos, ensure_ascii=False, indent=4)
    return json_str

In [ ]:
docx_pathR

In [ ]:
F = JSONTipoR(docx_pathR)

In [ ]:
print(F)

In [ ]:
r = 'AI-Redações/RedaçõesDownloaded/' #modificar el algoritmo si es q no hay comentarios
for i in archivos_filtradosR:
    print(i["id"],i["archivo"])
    i["JSON"] = JSONTipoR(r+str(i["id"])+"/"+str(i["archivo"]))

In [ ]:
diccionario = {}
Lid,Larchivo,Ltexto,Lcomentario,Lpalabras_corregidas = [],[],[],[],[]
tabla = pd.DataFrame()
for j in range(len(archivos_filtradosR)):
    data = json.loads(archivos_filtradosR[j]['JSON'])
    #data["COMENTARIOS"]
    for i in range(len(data["COMENTARIOS"])):
        print(data["COMENTARIOS"][i][1])
        #tabla["texto"] = [data["COMENTARIOS"][i][1]]
        #texto = data["COMENTARIOS"][i][1]
        Lid.append(archivos_filtradosR[j]["id"])
        Larchivo.append(archivos_filtradosR[j]['archivo'])
        Ltexto.append(data["COMENTARIOS"][i][1])
        Lcomentario.append(data["COMENTARIOS"][i][0])
        Lpalabras_corregidas.append(data["PALABRAS_CORREGIDAS"])
        #comentario = data["COMENTARIOS"][i][0]
        print(data["COMENTARIOS"][i][0])
        #diccionario["texto"] = texto
        #diccionario["comentario"] = comentario
        #tabla["comentario"] = [data["COMENTARIOS"][i][0]]

In [ ]:
datos = {
    "id":Lid,
    "archivo":Larchivo,
    "texto": Ltexto,
    "comentario":Lcomentario,
    "palabras_corregidas":Lpalabras_corregidas
}
df = pd.DataFrame(datos)

In [ ]:
df

In [ ]:
datos = {
    "id":Lid,
    "archivo":Larchivo,
    "texto": Ltexto,
    "comentario":Lcomentario,
    "palabras_corregidas":Lpalabras_corregidas
}
df = pd.DataFrame(datos)

In [ ]:
df

In [ ]:
#del df["palabras_corregidas"]

In [ ]:
df.to_excel("archivoR.xlsx", index=False)

# CORRECION

In [42]:
docx_pathZ = 'AI-Redações/RedaçõesDownloaded/R1/DIOGO DAVID SÁNCHEZ LIMA Z.docx'
docx_path1Z = 'AI-Redações/RedaçõesDownloaded/R1/Davi Natael da Cunha  ZZ.docx'
docx_path2Z = 'AI-Redações/RedaçõesDownloaded/R1/Carallow ZZ.docx'

In [43]:
docx_pathZ

'AI-Redações/RedaçõesDownloaded/R1/DIOGO DAVID SÁNCHEZ LIMA Z.docx'

In [44]:
texto_LIMPIO = extraer_texto_LIMPIO(docx_pathZ) #SIN NOTE Y TEXT
texto_LIMPIO = texto_LIMPIO.split('\n')
texto_LIMPIO = [elemento for elemento in texto_LIMPIO if elemento != ""] 
texto_LIMPIO = [elemento.replace("\xa0", "") for elemento in texto_LIMPIO]
texto_LIMPIO = [elemento for elemento in texto_LIMPIO if elemento != ""]

In [45]:
a = extraer_texto_en_parrafosTOTAL_SINEXCLUIDAS(docx_pathZ)
a = [elemento for elemento in a if elemento != ""] 
a = [elemento.replace("\xa0", "") for elemento in a]
a = [elemento for elemento in a if elemento != ""] 

b = todo(docx_pathZ)
#com = obtener_comentari

In [46]:
a

['DIOGO DAVID SÁNCHEZ LIMA',
 'dgsp3000ac@gmail.com',
 'R1N022',
 'THC: TRANSTORNO DO HOMEM PELO CELULAR',
 ' Dependência, palavra que costumava ser usada em diálogos sobre drogas e jogos de azar, hoje pode também ser utilizada quando se trata sobre as novas tecnologias esmartphonese a maneira como esse vício tornou-se prejudicial à sociedade .',
 ' Segundo O Globo, o número de pessoas com problemas na coluna cervical aumentou em até 40%, sendo que a maioria desses casos envolve jovens que possuem problemas de postura por usar constantemente os celulares de maneira incorreta.',
 ' Outro mal dessas tecnologias é que o uso desses dispositivos não agride só a saúde do usuário como também põe em risco a vida de terceiros. De acordo com a Abramet, são cerca de 54.000 mortes por ano no Brasil devido à utilização desses aparelhos no trânsito.',
 ' O grande responsável por todos esses problemas, não é única e exclusivamente o celular; o usuário do aparelho também carrega grande parte da culpa,

In [47]:
b

['Evaluation Only. Created with Aspose.Words. Copyright 2003-2023 Aspose Pty Ltd.\r\n',
 'DIOGO DAVID SÁNCHEZ LIMA\r\n',
 'dgsp3000ac@gmail.com\r\n',
 'R1N022\r\n',
 '\xa0\r\n',
 'THC: TRANSTORNO DO HOMEM PELO CELULAR\r\n',
 '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 Dependência, palavra que costumava ser usada em diálogos sobre drogas e jogos de azar, hoje pode também ser utilizada quando se trata sobre as novas tecnologias e\xa0smartphones\xa0e a maneira comocom que esse vício tornou-seé prejudicial àna sociedade atual\r\nBom início.\r\n.\r\n',
 'Bom início.\r\n',
 '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 Segundo O Globo, o número de pessoas com problemas na coluna cervical aumentouaram em até 40%, sendo que a maioria desses casos envolvem jovens que possuem problemas de postura por usar constantemente os celulares de maneira incorreta.\r\n',
 '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 Outro malu dessas tecnologias é que, o uso desses dispositivos não agridem\r\nVocê erra o tempo inteiro na conco

In [48]:
com = extraer_comentarios(docx_pathZ)
#PREPROCESAMIENTO
mi_lista_limpia1 = [elemento.replace("\xa0", " ") for elemento in b]
mi_lista_limpia1 = [elemento.replace("\r\n", "") for elemento in mi_lista_limpia1]
mi_lista_limpia1 = [elemento for elemento in mi_lista_limpia1 if elemento != ""]
mi_lista_limpia1 = [elemento for elemento in mi_lista_limpia1 if elemento != " "]
mi_lista_limpia1 = mi_lista_limpia1[1:-1]
resultado = [elemento for elemento in mi_lista_limpia1 if elemento not in com]

In [40]:
#com = extraer_comentarios(docx_pathZ)


In [50]:
resultado

['DIOGO DAVID SÁNCHEZ LIMA',
 'dgsp3000ac@gmail.com',
 'R1N022',
 'THC: TRANSTORNO DO HOMEM PELO CELULAR',
 '          Dependência, palavra que costumava ser usada em diálogos sobre drogas e jogos de azar, hoje pode também ser utilizada quando se trata sobre as novas tecnologias e smartphones e a maneira comocom que esse vício tornou-seé prejudicial àna sociedade atualBom início..',
 '          Segundo O Globo, o número de pessoas com problemas na coluna cervical aumentouaram em até 40%, sendo que a maioria desses casos envolvem jovens que possuem problemas de postura por usar constantemente os celulares de maneira incorreta.',
 '          Outro malu dessas tecnologias é que, o uso desses dispositivos não agridemVocê erra o tempo inteiro na concordância. só a saúde do usuário como também põem em risco a vida de terceiros. De, de acordo com a Aabramet, são cerca de 54.000 mortes por ano no Brasil devido àa utilização desses aparelhos no trânsito.',
 '          O grande responsável por t

In [51]:
for i in com:
    resultado3 = [elemento.replace(i,"") for elemento in resultado]

In [52]:
resultado3

['DIOGO DAVID SÁNCHEZ LIMA',
 'dgsp3000ac@gmail.com',
 'R1N022',
 'THC: TRANSTORNO DO HOMEM PELO CELULAR',
 '          Dependência, palavra que costumava ser usada em diálogos sobre drogas e jogos de azar, hoje pode também ser utilizada quando se trata sobre as novas tecnologias e smartphones e a maneira comocom que esse vício tornou-seé prejudicial àna sociedade atualBom início..',
 '          Segundo O Globo, o número de pessoas com problemas na coluna cervical aumentouaram em até 40%, sendo que a maioria desses casos envolvem jovens que possuem problemas de postura por usar constantemente os celulares de maneira incorreta.',
 '          Outro malu dessas tecnologias é que, o uso desses dispositivos não agridemVocê erra o tempo inteiro na concordância. só a saúde do usuário como também põem em risco a vida de terceiros. De, de acordo com a Aabramet, são cerca de 54.000 mortes por ano no Brasil devido àa utilização desses aparelhos no trânsito.',
 '          O grande responsável por t

In [53]:
un = a[-(len(a)-len(texto_LIMPIO)):]
nota = un[-1]
conclusion = un[:-1]
#print(nota), print(conclusion)

#NOME
nome = texto_LIMPIO[0].replace("Nome: ", "").replace("Nome:", "").replace("Aluno: ", "").replace("Aluno:", "")
del texto_LIMPIO[0]
#CORREO
correo = texto_LIMPIO[0].replace("E-mail: ", "").replace("E-mail:", "").replace("Email: ", "").replace("Email:", "").replace("\xa0", "")
del texto_LIMPIO[0]
#CODIGO
codigo = texto_LIMPIO[0].replace("Código de identificação: ", "").replace("Código de identificação:", "")
del texto_LIMPIO[0]
#NOTA

In [54]:
 #nota = texto_LIMPIO[-1].replace("NOTA: ", "").replace("nota: ", "").replace("Nota ", "")
 #del texto_LIMPIO[-1]
comentarios_y_texto = obtener_comentarios_y_texto_asociado(docx_pathZ)
comentarios_texto = [(tupla[0].replace('\x05', ''), tupla[1]) for tupla in comentarios_y_texto]
 
t2 = a[:len(texto_LIMPIO)]
t2 = t2[3:]
 
resultado1 = resultado[:len(texto_LIMPIO)]
resultado1 = resultado1[3:]
 
cuerpo = t2

In [55]:
# OBTENER PALABRAS COMPLETA CON ERROR
PALABRAS_CON_ERROR = []
for s1, s2 in zip(resultado1, t2):
    palabras_s1 = s1.split()
    palabras_s2 = s2.split()
    palabras_restantes = [palabra for palabra in palabras_s1 if palabra not in palabras_s2]
    frase_resultante = ' '.join(palabras_restantes)
    PALABRAS_CON_ERROR.extend(frase_resultante.split())

#print(PALABRAS_CON_ERROR) 

# OBTENER PALABRAS LIMPIAS CORREGIDAS
PALABRAS_CORREGIDO = []
for s1, s2 in zip(t2, resultado1):
    palabras_s1 = s1.split()
    palabras_s2 = s2.split()
    palabras_restantes = [palabra for palabra in palabras_s1 if palabra not in palabras_s2]
    frase_resultante = ' '.join(palabras_restantes)
    PALABRAS_CORREGIDO.extend(frase_resultante.split())

#print(PALABRAS_CORREGIDO) 
palabras_corregidas = [PALABRAS_CON_ERROR,PALABRAS_CORREGIDO]

In [56]:
PALABRAS_CON_ERROR

['smartphones',
 'comocom',
 'tornou-seé',
 'àna',
 'atualBom',
 'início..',
 'aumentouaram',
 'envolvem']

In [57]:
PALABRAS_CORREGIDO

['esmartphonese', 'como', 'tornou-se', 'à', '.', 'aumentou', 'envolve']

In [ ]:
#aqui

In [24]:
comentarios_y_texto = obtener_comentarios_y_texto_asociado(docx_pathZ)

In [26]:
comentarios_y_texto

[('\x05Bom início.', 'atual'),
 ('\x05Você erra o tempo inteiro na concordância.', 'agridem'),
 ('\x05Claro que o aparelho não tem culpa e sim quem o utiliza.',
  'O grande responsável por todas essestodos esses problemas, não é única e exclusivamente o celular;, o usuário do aparelho também carrega grande parte da culpa, pois se esse utilizasse de maneira adequada o dispositivo, não traria nenhum risco.'),
 ('\x05Não é bem verdade o que você diz',
  'nas escolas e nas universidades, onde estáa concentrada a maior parte dos usuários desses dispositivos móveis')]

In [25]:
comentarios_texto = [(tupla[0].replace('\x05', ''), tupla[1]) for tupla in comentarios_y_texto]

In [29]:
[(tupla[0].replace('\x05', ''), tupla[1]) for tupla in comentarios_y_texto]

[('Bom início.', 'atual'),
 ('Você erra o tempo inteiro na concordância.', 'agridem'),
 ('Claro que o aparelho não tem culpa e sim quem o utiliza.',
  'O grande responsável por todas essestodos esses problemas, não é única e exclusivamente o celular;, o usuário do aparelho também carrega grande parte da culpa, pois se esse utilizasse de maneira adequada o dispositivo, não traria nenhum risco.'),
 ('Não é bem verdade o que você diz',
  'nas escolas e nas universidades, onde estáa concentrada a maior parte dos usuários desses dispositivos móveis')]

In [30]:
comentarios_texto

[('Bom início.', 'atual'),
 ('Você erra o tempo inteiro na concordância.', 'agridem'),
 ('Claro que o aparelho não tem culpa e sim quem o utiliza.',
  'O grande responsável por todas essestodos esses problemas, não é única e exclusivamente o celular;, o usuário do aparelho também carrega grande parte da culpa, pois se esse utilizasse de maneira adequada o dispositivo, não traria nenhum risco.'),
 ('Não é bem verdade o que você diz',
  'nas escolas e nas universidades, onde estáa concentrada a maior parte dos usuários desses dispositivos móveis')]

In [35]:
coment = []
for i in comentarios_texto:
    print(i[0])
    coment.append(i[0])

Bom início.
Você erra o tempo inteiro na concordância.
Claro que o aparelho não tem culpa e sim quem o utiliza.
Não é bem verdade o que você diz


In [36]:
coment

['Bom início.',
 'Você erra o tempo inteiro na concordância.',
 'Claro que o aparelho não tem culpa e sim quem o utiliza.',
 'Não é bem verdade o que você diz']

In [18]:
mi_lista_limpia1

['DIOGO DAVID SÁNCHEZ LIMA',
 'dgsp3000ac@gmail.com',
 'R1N022',
 'THC: TRANSTORNO DO HOMEM PELO CELULAR',
 '          Dependência, palavra que costumava ser usada em diálogos sobre drogas e jogos de azar, hoje pode também ser utilizada quando se trata sobre as novas tecnologias e smartphones e a maneira comocom que esse vício tornou-seé prejudicial àna sociedade atualBom início..',
 'Bom início.',
 '          Segundo O Globo, o número de pessoas com problemas na coluna cervical aumentouaram em até 40%, sendo que a maioria desses casos envolvem jovens que possuem problemas de postura por usar constantemente os celulares de maneira incorreta.',
 '          Outro malu dessas tecnologias é que, o uso desses dispositivos não agridemVocê erra o tempo inteiro na concordância. só a saúde do usuário como também põem em risco a vida de terceiros. De, de acordo com a Aabramet, são cerca de 54.000 mortes por ano no Brasil devido àa utilização desses aparelhos no trânsito.',
 'Você erra o tempo in

In [ ]:
#docx_pathZ

In [77]:
from docx import Document

def extraer_palabras_tachadas(docx_path):
    doc = Document(docx_path)

    palabras_tachadas = []

    for paragraph in doc.paragraphs:
        for run in paragraph.runs:
            if run.font.strike:
                palabras_tachadas.append(run.text)

    return palabras_tachadas

# Ruta de tu documento Word docx_pathZ
ruta_documento = docx_pathZ

# Llamada a la función
palabras_tachadas = extraer_palabras_tachadas(ruta_documento)

# Imprimir las palabras tachadas
for palabra in palabras_tachadas:
    print(palabra)


In [78]:
palabras_tachadas

[]

In [62]:
docx_pathZ

'AI-Redações/RedaçõesDownloaded/R1/DIOGO DAVID SÁNCHEZ LIMA Z.docx'

In [64]:
!pip install docx2txt

  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3963 sha256=145ea8e6c52df2844abe5db8b514c5d151d5befc4a3c18ad68896d49e4376de5
  Stored in directory: /home/andres/.cache/pip/wheels/b7/20/b2/473e3aea9a0c0d3e7b2f7bd81d06d0794fec12752733d1f3a8
Successfully built docx2txt


In [69]:
from docx import Document
from docx.shared import Pt

def extraer_palabras_tachadas(docx_path):
    document = Document(docx_path)

    palabras_tachadas = []

    for paragraph in document.paragraphs:
        for run in paragraph.runs:
            # Verificar si la fuente tiene el atributo strike y size menor a 1
            if run.font.strike and run.font.size < Pt(20):
                palabras_tachadas.append(run.text)

    return palabras_tachadas

# Ruta al archivo Word
archivo_word = docx_pathZ

# Llamada a la función para extraer palabras tachadas
palabras_tachadas = extraer_palabras_tachadas(docx_pathZ)

# Imprimir las palabras tachadas
print("Palabras tachadas:")
for palabra in palabras_tachadas:
    print(palabra)

Palabras tachadas:


In [72]:
docx_pathZ

'AI-Redações/RedaçõesDownloaded/R1/DIOGO DAVID SÁNCHEZ LIMA Z.docx'

In [74]:
todo(docx_pathZ)

['Evaluation Only. Created with Aspose.Words. Copyright 2003-2023 Aspose Pty Ltd.\r\n',
 'DIOGO DAVID SÁNCHEZ LIMA\r\n',
 'dgsp3000ac@gmail.com\r\n',
 'R1N022\r\n',
 '\xa0\r\n',
 'THC: TRANSTORNO DO HOMEM PELO CELULAR\r\n',
 '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 Dependência, palavra que costumava ser usada em diálogos sobre drogas e jogos de azar, hoje pode também ser utilizada quando se trata sobre as novas tecnologias e\xa0smartphones\xa0e a maneira comocom que esse vício tornou-seé prejudicial àna sociedade atual\r\nBom início.\r\n.\r\n',
 'Bom início.\r\n',
 '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 Segundo O Globo, o número de pessoas com problemas na coluna cervical aumentouaram em até 40%, sendo que a maioria desses casos envolvem jovens que possuem problemas de postura por usar constantemente os celulares de maneira incorreta.\r\n',
 '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 Outro malu dessas tecnologias é que, o uso desses dispositivos não agridem\r\nVocê erra o tempo inteiro na conco

In [76]:
extraer_texto_LIMPIO(docx_pathZ)

'DIOGO DAVID SÁNCHEZ LIMA\ndgsp3000ac@gmail.com\nR1N022\n\xa0\nTHC: TRANSTORNO DO HOMEM PELO CELULAR\n\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 Dependência, palavra que costumava ser usada em diálogos sobre drogas e jogos de azar, hoje pode também ser utilizada quando se trata sobre as novas tecnologias e\xa0smartphones\xa0e a maneira  esse vício  prejudicial  sociedade .\n\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 Segundo O Globo, o número de pessoas com problemas na coluna cervical aument em até 40% sendo que a maioria desses casos envolve jovens que possuem problemas de postura por usar constantemente os celulares de maneira incorreta.\n\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 Outro ma dessas tecnologias é que o uso desses dispositivos não agride só a saúde do usuário como também põe em risco a vida de terceiros acordo com a bramet são cerca de 54.000 mortes por ano no Brasil devido  utilização desses aparelhos no trânsito.\n\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 O grande responsável por  problemas,

In [75]:
extraer_texto_en_parrafosTOTAL_SINEXCLUIDAS(docx_pathZ)

['DIOGO DAVID SÁNCHEZ LIMA',
 'dgsp3000ac@gmail.com',
 'R1N022',
 '\xa0',
 'THC: TRANSTORNO DO HOMEM PELO CELULAR',
 '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 Dependência, palavra que costumava ser usada em diálogos sobre drogas e jogos de azar, hoje pode também ser utilizada quando se trata sobre as novas tecnologias e\xa0smartphones\xa0e a maneira como esse vício tornou-se prejudicial à sociedade .',
 '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 Segundo O Globo, o número de pessoas com problemas na coluna cervical aumentou em até 40%, sendo que a maioria desses casos envolve jovens que possuem problemas de postura por usar constantemente os celulares de maneira incorreta.',
 '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 Outro mal dessas tecnologias é que o uso desses dispositivos não agride só a saúde do usuário como também põe em risco a vida de terceiros. De acordo com a Abramet, são cerca de 54.000 mortes por ano no Brasil devido à utilização desses aparelhos no trânsito.',
 '\xa0\xa0\xa0\xa0\xa0\xa

In [79]:
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)

In [81]:
df.head(), df.shape

(                                                   0  1
 0  a stirring , funny and finally transporting re...  1
 1  apparently reassembled from the cutting room f...  0
 2  they presume their audience wo n't sit still f...  0
 3  this is a visually stunning rumination on love...  1
 4  jonathan parker 's bartleby should have been t...  1,
 (6920, 2))

In [82]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

# Cargar el tokenizador y el modelo preentrenado para análisis de sentimiento
model_name = 'nlptown/bert-base-multilingual-uncased-sentiment'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

# Crear una función para realizar el análisis de sentimiento
def analisis_sentimiento(texto):
    tokens = tokenizer(texto, return_tensors='pt')
    resultado = model(**tokens)
    probabilidad = resultado.logits.softmax(dim=1)
    sentimiento = probabilidad.argmax().item()
    return sentimiento

# Ejemplo de uso
texto_ejemplo = "Estoy muy contento de poder utilizar BERT para análisis de sentimiento."
sentimiento_resultante = analisis_sentimiento(texto_ejemplo)

# Imprimir el resultado
print(f"Sentimiento resultante: {sentimiento_resultante}")


Sentimiento resultante: 4


In [ ]:
df['Sentimiento2'] = df['Texto'].apply(analisis_sentimiento)